In [1]:
from itertools import combinations
import time
import re
import math
import random
from z3 import *

from typing import Tuple, Dict

import sys
BASE_PATH = os.path.join(sys.path[0].split('cdmo1-project')[0], 'cdmo1-project')
sys.path.append(os.path.join(BASE_PATH, 'utils'))
from problem import ProblemInstance, parse_problem_file
from solution import SolutionInstance, Circuit
from initial_solution import construct_initial_solution
from z3_utils import exactly_one, index_orders, at_least_one

In [2]:
INSTANCE_NUM = 20

inst = parse_problem_file(os.path.join(BASE_PATH, f'instances/ins-{INSTANCE_NUM}.txt'))
initial_sol = construct_initial_solution(inst)

In [3]:
def get_cw(circ:int):
    return inst.circuits[circ].w

def get_ch(circ:int):
    return inst.circuits[circ].h

def max_z3(vars):
    max = vars[0]
    for v in vars[1:]:
        max = If(v > max, v, max)
    return max

def cumulative_z3(start_times, durations, res_requirements, bound):
    '''
    Cumulative constraint for Z3: we iterate over all possible times within bound
    and for each time we check which tasks are involved by considering the start and
    end times for each task. For all involved tasks, we sum their resource requirements
    and constraint it to be below the fixed bound.
    '''
    return [Sum([                                                   # Sum over times in bound
                If(start_times[i] <= t, 1, 0) *                     # For each task, if the time is after its starting time, return 1, 0 otherwise
                If(t < start_times[i] + durations[i], 1, 0) *       # Multiply the 1 or 0 with the variable representing whether the time is also before the start_time + duration of the task
                res_requirements[i]                                 # If both indices are 1, the task is to be considered involved, therefore we sum the resource requirements with all others for that time
                for i in range(len(start_times))]) <= bound         # The sum of resource requirements must be kept below the bound threshold.
            for t in range(bound)]

def lex_lessex_z3(vars1, vars2):
    constraints = [vars1[0] <= vars2[0]]
    for i in range(1, len(vars1)-1):
        constraints.append(Implies(vars1[i] == vars2[i], vars1[i+1] <= vars2[i+1]))
    return And(constraints)


In [8]:
## Constants
W = inst.wg
n = inst.n
low_h = math.floor(sum([get_ch(circ)*get_cw(circ) for circ in range(n)]) / W)
max_h = max([circ.y0 + circ.h for circ in initial_sol.circuits])

## Optimizer
opt = Optimize()
opt.set(timeout=5*60*1000)

## Variables
cx = [Int(f'x_{circ}') for circ in range(n)]
cy = [Int(f'y_{circ}') for circ in range(n)]
cw = [Int(f'cw_{circ}') for circ in range(n)]
ch = [Int(f'ch_{circ}') for circ in range(n)]
transl_pos = [Int(f'tr_{circ}') for circ in range(n)]
h = Int(f'h')

## Domains
for circ in range(n):
    opt.add(And(0 <= cx[circ], cx[circ] <= W - get_cw(circ)))
    opt.add(And(0 <= cy[circ], cy[circ] <= h - get_ch(circ)))
    opt.add(And(low_h <= h, h <= max_h))
    opt.add(transl_pos[circ] == cy[circ] * W + cx[circ])

# All translated positions must be different
opt.add(Distinct(transl_pos))
# One circuit must be at 0
exactly_one(opt, [transl_pos[circ] == 0 for circ in range(n)])
    
## Constraints
# Bidimensional No-Overlap
for ci, cj, in combinations(range(n), 2):
    opt.add(Or(
        cx[ci] + get_cw(ci) <= cx[cj],
        cx[cj] + get_cw(cj) <= cx[ci],
        cy[ci] + get_ch(ci) <= cy[cj],
        cy[cj] + get_ch(cj) <= cy[ci]
    ))

# Cumulative constraint
opt.add(cumulative_z3(cy, [get_ch(circ) for circ in range(n)],
    [get_cw(circ) for circ in range(n)], W))
#opt.add(cumulative_z3(cx, [get_cw(circ) for circ in range(n)],
#    [get_ch(circ) for circ in range(n)], low_h))

## Symmetry breaking
opt.add([
    lex_lessex_z3(transl_pos, [cy[c] * W + (W - cx[c] - get_cw(c)) for c in range(n)]),
    lex_lessex_z3(transl_pos, [(h - cy[c] * get_ch(c)) * W + cx[c] for c in range(n)]),
    lex_lessex_z3(transl_pos, [(h - cy[c] - get_ch(c)) * W + (W - cx[c] - get_cw(c)) for c in range(n)])
])

## Definition of the height variable
opt.add(h == max_z3([cy[circ] + get_ch(circ) for circ in range(n)]))

# Minimization problem
opt.minimize(h)

In [9]:
def obtain_solution(opt, verbose=False) -> Tuple[SolutionInstance, Dict[str, float]]:
    times = {
        'check_time': 0.0,
        'model_time': 0.0
    }
    start_check = time.time()
    check = opt.check()
    end_check = time.time()
    times['check_time'] = end_check - start_check
    if check == sat:
        # If SAT, construct the solution
        start_get_true_vars = time.time()
        sol = opt.model()
        end_get_true_vars = time.time()
        times['model_time'] = end_get_true_vars - start_get_true_vars
        if verbose:
            print(sol)
        assignments_x = [int(sol[var].as_long()) 
            for circ in range(n)
            for var in sol if re.match(f'x_{circ}$', str(var)) ]
        assignments_y = [int(sol[var].as_long()) 
            for circ in range(n)
            for var in sol if re.match(f'y_{circ}$', str(var)) ]
        sol_h = int(sol[h].as_long())
        if verbose:
            print(assignments_x, assignments_y)
            print("Solution h: {}".format(sol_h))
        solution = SolutionInstance(
            W, sol_h, n,
            [ Circuit(get_cw(i), get_ch(i), assignments_x[i], assignments_y[i]) 
                for i in range(n) ]
        )
        return solution, times
    else:
        print(check)
        

In [10]:
solution, model_time = obtain_solution(opt)
solution.draw()